### Dash plotly for Interactive visualization 

In [1]:
# from the solution given by chatgpt
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

In [2]:
# Sample DataFrame (replace with your actual spacex_df)
spacex_df = pd.read_csv('spacex_launch_geo.csv')
spacex_df.head(5)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [3]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [4]:
# Prepare options for the dropdown
launch_sites = spacex_df['Launch Site'].unique()
options = [{'label': site, 'value': site} for site in launch_sites]
options.insert(0, {'label': 'All', 'value': 'All'})  # Default option
options

[{'label': 'All', 'value': 'All'},
 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}]

In [5]:
# Define payload range
min_payload = 0
max_payload = 10000

In [6]:
# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='site-dropdown',
        options=options,
        value='All',  # Default value
        clearable=False,
        placeholder='Select a Launch Site'
    ),
    dcc.Graph(id='success-pie-chart'),  # Pie chart placeholder
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        value=[min_payload, max_payload],  # Current selected range
        marks={i: str(i) for i in range(0, max_payload + 1, 1000)}  # Marks for the slider
    ),
    dcc.Graph(id='success-payload-scatter-chart')  # Scatter plot placeholder
])


In [7]:
# Callback to update the pie chart based on selected dropdown value
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    # If "All" is selected, use the entire DataFrame
    if selected_site == 'All':
        success_counts = spacex_df['class'].value_counts()
        labels = ['Failed', 'Success']
        values = [success_counts.get(0, 0), success_counts.get(1, 0)]
    else:
        # Filter DataFrame for the selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_counts = filtered_df['class'].value_counts()
        labels = ['Failed', 'Success']
        values = [success_counts.get(0, 0), success_counts.get(1, 0)]

    # Create a pie chart
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
    fig.update_layout(title_text=f'Success Counts for {selected_site}')

    return fig


In [8]:
# Callback to update the scatter chart based on selected dropdown value and payload range
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_chart(selected_site, payload_range):
    # Filter DataFrame based on the selected site and payload range
    filtered_df = spacex_df.copy()
    
    if selected_site != 'All':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    
    # Further filter by payload range
    filtered_df = filtered_df[
        (filtered_df['Payload Mass (kg)'] >= payload_range[0]) & 
        (filtered_df['Payload Mass (kg)'] <= payload_range[1])
    ]

    # Create a scatter plot
    fig = px.scatter(
        filtered_df, 
        x='Payload Mass (kg)', 
        y='class', 
        color='Booster Version',
        title='Payload vs. Mission Outcome',
        labels={'class': 'Mission Outcome (1=Success, 0=Failure)', 'Payload Mass (kg)': 'Payload Mass (kg)'}
    )
    
    return fig

In [9]:
if __name__ == '__main__':
    app.run_server(debug=True)